In [1]:
import pandas as pd
import string
from Preprocessing import preprocessing

In [2]:
# delete output file if already exists
!rm data/processed/output_POS.txt

# call preprocessing to initiate preprocessing of all the books
preprocessing()

Done processing: Devices And Desires
Done processing: The Sandcastle
Done processing: Destination Unknown
Done processing: Elephants Can Remember
Done processing: Death Comes To Pemberley
Done processing: Cover Her Face
Done processing: Jacksons Dilemma
Done processing: And Then There Were None
Done processing: The Black Prince


In [3]:
#TODO add year for each book

book_info_df = pd.read_csv("data/processed/output_POS.txt", 
                           delimiter='|',
                           index_col=False,
                           quoting=3, 
                           encoding='utf-8')

print(book_info_df.shape)
book_info_df

(9, 72)


,Author,Title,book_order,total_words,avg_sentence_size,!,#,"""",%,$,...,VBD,VBG,VBP,VBN,WDT,VBZ,WRB,WP$,WP,Unnamed: 71
0,P.D. James,Devices And Desires,second,157218,14.983132,0.001079,0,0.165648,0,0.000000,...,0.066564,0.015928,0.021824,0.022531,0.003304,0.011203,0.005318,0.000094,0.004860,NaN
1,Iris Murdoch,The Sandcastle,first,115763,13.898787,0.012219,0,0.206783,0,0.000000,...,0.081775,0.018358,0.010258,0.020354,0.005265,0.007518,0.004006,0.000118,0.004109,NaN
2,Agatha Christie,Destination Unknown,second,60478,10.253984,0.012498,0,0.282553,0,0.000000,...,0.060963,0.012452,0.029284,0.017914,0.002343,0.014655,0.004533,0.000038,0.005971,NaN
3,Agatha Christie,Elephants Can Remember,third,60026,11.163474,0.002634,0,0.284720,0,0.000000,...,0.063865,0.010610,0.039686,0.016994,0.003580,0.013776,0.005053,0.000052,0.009059,NaN
4,P.D. James,Death Comes To Pemberley,third,90974,19.237471,0.001940,0,0.128828,0,0.000000,...,0.066433,0.014996,0.012163,0.032467,0.004970,0.009582,0.005656,0.000184,0.004506,NaN
5,P.D. James,Cover Her Face,first,79347,13.633505,0.006820,0,0.153714,0,0.000000,...,0.073576,0.014477,0.015781,0.026823,0.003134,0.010517,0.004963,0.000145,0.005231,NaN
6,Iris Murdoch,Jacksons Dilemma,third,89939,17.450330,0.024277,0,0.189054,0,0.000000,...,0.074687,0.021951,0.013830,0.021275,0.002750,0.009363,0.006550,0.000082,0.005517,NaN
7,Agatha Christie,And Then There Were None,first,54108,9.090726,0.024591,0,0.185769,0,0.000062,...,0.073138,0.013511,0.017264,0.018840,0.001952,0.013285,0.003753,0.000030,0.005569,NaN
8,Iris Murdoch,The Black Prince,second,137257,13.419730,0.010621,0,0.274566,0,0.000000,...,0.054606,0.017702,0.021827,0.017068,0.003288,0.016542,0.005219,0.000054,0.004914,NaN


In [4]:
book_info_df.to_csv("data/processed/output_POS.csv")